In [27]:
import os
os.chdir('/notebooks/src/CIFAR-10/')

In [28]:
import keras
from keras.layers import Input, Convolution2D, Dense, Activation, Flatten,merge
from keras.layers import  MaxPooling2D, Dropout, LocallyConnected2D
from keras.models import Model, load_model
from keras.optimizers import SGD
from lib.load_images import load_data
import numpy as np
import pandas as pd


from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten, merge
from keras.models import Sequential, Model
from keras.activations import relu, softmax
from keras.layers.advanced_activations import LeakyReLU
from keras import layers as l

In [29]:
(X_train, y_train), (X_test, y_test) = load_data()

In [30]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


('X_train shape:', (50000, 32, 32, 3))
(50000, 'train samples')
(10000, 'test samples')


In [31]:
batch_size = 32
num_classes = 10
epochs = 200
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cnn_3_aug.h5'

In [32]:
cnn = Sequential([
    Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]),
    
    Activation('relu'),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(.25),
    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
])

# RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

cnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [33]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
__________

In [34]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)
# Fit the model on the batches generated by datagen.flow().
cnn.fit_generator(datagen.flow(X_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

Epoch 1/200
1562/1562 [==============================] - 411s - loss: 1.8561 - acc: 0.3133 - val_loss: 1.5386 - val_acc: 0.4463
Epoch 2/200
1562/1562 [==============================] - 550s - loss: 1.5511 - acc: 0.4340 - val_loss: 1.4160 - val_acc: 0.4961
Epoch 3/200
1562/1562 [==============================] - 550s - loss: 1.1222 - acc: 0.6045 - val_loss: 0.9600 - val_acc: 0.6653
Epoch 9/200
1562/1562 [==============================] - 549s - loss: 1.0467 - acc: 0.6313 - val_loss: 0.9245 - val_acc: 0.6779
Epoch 11/200
1562/1562 [==============================] - 549s - loss: 0.8785 - acc: 0.6966 - val_loss: 0.8015 - val_acc: 0.7257
Epoch 21/200
1562/1562 [==============================] - 546s - loss: 0.8744 - acc: 0.6975 - val_loss: 0.7372 - val_acc: 0.7469
Epoch 22/200
1562/1562 [==============================] - 550s - loss: 0.8639 - acc: 0.7015 - val_loss: 0.7381 - val_acc: 0.7439
Epoch 23/200
1562/1562 [==============================] - 550s - loss: 0.8632 - acc: 0.7033 - val_los

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
cnn.save(model_path)

In [ ]:
# accuracy = history.history['acc']
# val_accuracy = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# with open('accuracy_3_CNN_fchallot_aug.pickle', 'wb') as handle:
#     pickle.dump(accuracy, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pickle', 'rb') as handle:
# #     b = pickle.load(handle)

# with open('val_accuracy_3_CNN_fchallot_aug.pickle', 'wb') as handle:
#     pickle.dump(val_accuracy, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pickle', 'rb') as handle:
# #     b = pickle.load(handle)
# with open('loss_3_CNN_fchallot_aug.pickle', 'wb') as handle:
#     pickle.dump(loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pickle', 'rb') as handle:
# #     b = pickle.load(handle)

# with open('val_loss_3_CNN_fchallot_aug.pickle', 'wb') as handle:
#     pickle.dump(val_loss, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pickle', 'rb') as handle:
# #     b = pickle.load(handle)